<a href="https://colab.research.google.com/github/MGabr/fm-next-songs-rec/blob/master/AOTM_2011_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="6">**AOTM-2011 small**</font>

# Data loading and preprocessing

This notebook creates a small preprocessed subset of the AOTM-2011 dataset, split into train, test and test query dataset and saves the datasets as csv files.

These csv files can loaded as Spark dataframes and used for integration tests of Spark recommenders. The preprocessing transforms each string column of labels to a column of label indices similar to Spark's *StringIndexer*. Unknown labels or NaNs are transformed to the largest label index for compatibility with Spark's *OneHotEncoderEstimator*.

In [0]:
import os

import pandas as pd
import numpy as np

from google.colab import drive, files

In [0]:
os.chdir("/")
drive.mount("/drive", force_remount=True)

!ls "/drive/My Drive/datasets"
!gunzip -k -c "/drive/My Drive/datasets/aotm2011_playlists.json.gz" > "aotm2011_playlists.json"

os.chdir("/drive/My Drive/")
if not os.path.exists("models"):
  os.mkdir("models")
if not os.path.exists("models/aotm-2011"):
  os.mkdir("models/aotm-2011")
os.chdir("/")

Mounted at /drive
aotm2011_playlists.json.gz  lastfm-dataset-1K.tar.gz  msd_summary_file.h5


In [0]:
playlists_filename = "aotm2011_playlists.json"
msd_filename = "/drive/My Drive/datasets/msd_summary_file.h5"
models_dirname = "/drive/My Drive/models/aotm-2011/"

In [0]:
df = pd.read_json(playlists_filename, orient="records")

df.head()

,category,filtered_lists,playlist,timestamp,mix_id,user
0,Mixed Genre,"[[SOFDPDC12A58A7D198], [SOPIEQP12A8C13F268, SO...","[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,89567,"{'mixes_posted': '23', 'member_since': '2004-0..."
1,Theme,"[[SOQOVCB12A58A7E83B], [SOGKLYN12AB017F155, SO...","[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,117124,"{'mixes_posted': '7', 'member_since': '2006-11..."
2,Theme,[[SOFIQBR12A6D4F78C6]],"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,97418,"{'mixes_posted': '93', 'member_since': '2003-1..."
3,Theme,"[[SORJUFZ12A67ADAF25, SOEELNN12AB018AC8D], [SO...","[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,23876,"{'mixes_posted': '8', 'member_since': '2002-03..."
4,Theme,"[[SOOUYTP12B0B8094B6, SOAJODS12A8C138A23], [SO...","[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,70694,"{'mixes_posted': '60', 'member_since': '2004-0..."


In [0]:
df["pid"] = df["mix_id"]
df["userid"] = df["user"].map(lambda u: u["name"])

# explode into sessions consecutive of tracks with MSD ids in playlists
s = df["filtered_lists"].apply(pd.Series)
id_vars = ["category", "pid", "timestamp", "userid"]
df = df[id_vars]
df = s.merge(df, left_index=True, right_index=True)
df = pd.melt(df, id_vars, var_name="sid", value_name="filtered_list")
df = df[~df["filtered_list"].isnull()]

df.head()

,category,pid,timestamp,userid,sid,filtered_list
0,Mixed Genre,89567,2005-03-27 10:53:00,pulmotor,0,[SOFDPDC12A58A7D198]
1,Theme,117124,2008-01-11 21:45:00,caterpillar,0,[SOQOVCB12A58A7E83B]
2,Theme,97418,2005-10-31 10:15:00,gobi,0,[SOFIQBR12A6D4F78C6]
3,Theme,23876,2002-03-20 17:59:00,Osmundson,0,"[SORJUFZ12A67ADAF25, SOEELNN12AB018AC8D]"
4,Theme,70694,2004-03-23 03:43:00,ManOSand,0,"[SOOUYTP12B0B8094B6, SOAJODS12A8C138A23]"


In [0]:
# explode sessions into tracks, keeping session ids
s = df["filtered_list"].apply(pd.Series)
id_vars = id_vars + ["sid"]
df = df[id_vars]
df = s.merge(df, left_index=True, right_index=True)
df = pd.melt(df, id_vars, var_name="varid", value_name="traid")
df = df[~df["traid"].isnull()]
id_vars = id_vars + ["traid"]
df = df[id_vars]

df.sort_values(["pid", "sid"], kind="mergesort", inplace=True)

del s

df.head()

,category,pid,timestamp,userid,sid,traid
34256,Rock/Pop,1,1998-01-22,Eli Lester,0,SOHVGOY12AF729EC76
130601,Rock/Pop,1,1998-01-22,Eli Lester,1,SORTBLM12A8C140261
590015,Rock/Pop,1,1998-01-22,Eli Lester,1,SOWENZL12AB0186BEE
221992,Rock/Pop,1,1998-01-22,Eli Lester,2,SOURMTG12A8C13B478
681406,Rock/Pop,1,1998-01-22,Eli Lester,2,SOCAXNF12A81356D4D


In [0]:
def with_msd_columns(df):
  msd_cols = ["artist_id", "release", "song_id", "artist_name", "title"]
  msd_df = pd.read_hdf(msd_filename, key="metadata/songs")[msd_cols]
  rename_map = {"artist_id": "artid", "release": "albid", "song_id": "traid",
                "artist_name": "artname", "title": "traname"}
  msd_df.rename(columns=rename_map, inplace=True)
  msd_df["albname"] = msd_df["albid"]
  msd_df["albid"] = msd_df["artid"] + " " + msd_df["albid"]
  
  msd_year_df = pd.read_hdf(msd_filename, key="musicbrainz/songs")[["year"]]
  msd_year_df.replace(0.0, np.nan, inplace=True)
  msd_df = pd.concat([msd_df, msd_year_df], axis=1)
  
  msd_df = msd_df.drop_duplicates("traid")
  msd_df = msd_df[msd_df["traid"].isin(df["traid"].drop_duplicates())]
  
  return df.join(msd_df.set_index("traid"), on="traid")

df = with_msd_columns(df)
df.head()

,category,pid,timestamp,userid,sid,traid,artid,albid,artname,traname,albname,year
34256,Rock/Pop,1,1998-01-22,Eli Lester,0,SOHVGOY12AF729EC76,ARBAN8S1187FB4D22F,ARBAN8S1187FB4D22F Ben Folds Five,Ben Folds Five,Philosophy,Ben Folds Five,1995.0
130601,Rock/Pop,1,1998-01-22,Eli Lester,1,SORTBLM12A8C140261,AR35E9F1187B9B4877,AR35E9F1187B9B4877 The Missing Years,John Prine,All the Best,The Missing Years,1991.0
590015,Rock/Pop,1,1998-01-22,Eli Lester,1,SOWENZL12AB0186BEE,ARVHCWL1187B9B8F60,ARVHCWL1187B9B8F60 B-Sides And Otherwise,Morphine,All Wrong (Album Version),B-Sides And Otherwise,1997.0
221992,Rock/Pop,1,1998-01-22,Eli Lester,2,SOURMTG12A8C13B478,ARPIRQD1187B9AE9CA,ARPIRQD1187B9AE9CA March,Michael Penn,No Myth,March,1989.0
681406,Rock/Pop,1,1998-01-22,Eli Lester,2,SOCAXNF12A81356D4D,ARVFF0Y1187B9AC3CA,ARVFF0Y1187B9AC3CA Crooked Rain Crooked Rain (...,Pavement,Camera,Crooked Rain Crooked Rain (Deluxe Edition),1994.0


In [0]:
os.chdir("/drive/My Drive/")

In [0]:
df_counts = df.groupby("traid")["pid"].count()

In [0]:
df_counts = df_counts.sort_values(ascending=False)

In [0]:
df_counts

traid
SOBBKGF12A8C1311EE    829
SOXUYUZ12A8C13A67E    666
SOPQJKW12A58A7AB38    665
SOGRCBF12A8C13233B    662
SOAKMDU12A8C1346A9    653
                     ... 
SOHPGQZ12A6D4F66A9      1
SOHPGPU12A58A7937A      1
SOQXQGZ12AB018981E      1
SOHPGHW12A6D4FB357      1
SOFYCCT12A6D4F5BF7      1
Name: pid, Length: 119894, dtype: int64

In [0]:
df_pids_small = df[df["traid"] == "SOBBKGF12A8C1311EE"]["pid"]

In [0]:
df_small = df[df["pid"].isin(df_pids_small)].copy()

In [0]:
df_small["pid"].nunique()

828

In [0]:
def with_prev_columns(df):
  gp = df.groupby("pid")
  df["prev_traid"] = gp["traid"].shift(1)
  df["prev_albid"] = gp["albid"].shift(1)
  df["prev_artid"] = gp["artid"].shift(1)
  df["prev_year"] = gp["year"].shift(1)
  df["prev_year"].fillna(0, inplace=True)
  df["prev_year"] = df["prev_year"].astype(int, copy=False)
  return df

df_small = with_prev_columns(df_small)
df_small.head()

,category,pid,timestamp,userid,sid,traid,artid,albid,artname,traname,albname,year,prev_traid,prev_albid,prev_artid,prev_year
42341,Mixed Genre,85,1998-01-30,Bob Paffile,0,SOBBKGF12A8C1311EE,AR1E9AW1187B9AC8F6,AR1E9AW1187B9AC8F6 Wave Of Mutilation: Best Of...,Pixies,Where Is My Mind?,Wave Of Mutilation: Best Of Pixies,1988.0,NaN,NaN,NaN,0
138375,Mixed Genre,85,1998-01-30,Bob Paffile,1,SOHXPRI12A6D4F94B0,AR0962A1187B9B43E9,AR0962A1187B9B43E9 Fight For Your Mind,Ben Harper,Burn One Down,Fight For Your Mind,NaN,SOBBKGF12A8C1311EE,AR1E9AW1187B9AC8F6 Wave Of Mutilation: Best Of...,AR1E9AW1187B9AC8F6,1988
229198,Mixed Genre,85,1998-01-30,Bob Paffile,2,SOWOEKH12A58A7E811,ARV4GYZ1187B9B8171,ARV4GYZ1187B9B8171 Hallowed Ground,Violent Femmes,Country Death Song,Hallowed Ground,1984.0,SOHXPRI12A6D4F94B0,AR0962A1187B9B43E9 Fight For Your Mind,AR0962A1187B9B43E9,0
309912,Mixed Genre,85,1998-01-30,Bob Paffile,3,SOOSNZX12D021B0CFE,ARW8F4B1187FB45EB4,ARW8F4B1187FB45EB4 40oz To Freedom,Sublime,5446 Thats My Number/ Ball And Chain,40oz To Freedom,NaN,SOWOEKH12A58A7E811,ARV4GYZ1187B9B8171 Hallowed Ground,ARV4GYZ1187B9B8171,1984
769326,Mixed Genre,85,1998-01-30,Bob Paffile,3,SOPWBHC12AF72A7B35,AR1E9AW1187B9AC8F6,AR1E9AW1187B9AC8F6 Debaser,Pixies,Gigantic,Debaser,1988.0,SOOSNZX12D021B0CFE,ARW8F4B1187FB45EB4 40oz To Freedom,ARW8F4B1187FB45EB4,0


# Evaluation split

In [0]:
def col2id_col(df, col, mappings, rvs_mappings):
  # ONLY THIS METHOD IS CHANGED,
  # UNKNOWN COLUMN VALUES ARE MAPPED TO NUM_VALS INSTEAD OF 0

  if col in mappings:
    df[col] = df[col].map(mappings[col])
    df[col].fillna(len(mappings[col]) - 1, inplace=True)
  else:
    df[col], rvs_mappings[col] = df[col].factorize()
    df[col] = df[col].replace(-1, len(rvs_mappings[col]))
    rvs_mappings[col] = rvs_mappings[col].insert(len(rvs_mappings[col]), None)
    mappings[col] = {col_val: i for i, col_val in enumerate(rvs_mappings[col])}

  df[col] = df[col].astype("int64", copy=False)


def df2id_df(df, id_cols, mappings, rvs_mappings):
  for id_col in id_cols:
    col2id_col(df, id_col, mappings, rvs_mappings)


def split_dfs2id_dfs(train_df, test_df=None, val_df=None, copy=True, cols=[],
                     same_col_pairs=[]):
  """
  Map columns of dataframes to one-hot indices.
  Test and validation dataframes will use the mappings of the train dataframe.
  Index 0 is used for missing and unknown (test and validation) column values.

  same_col_pairs are (src_col, same_col) pairs where same_col uses the same
  mappings as src_col.
  """
  id_mappings = dict()
  rvs_id_mappings = dict()
  
  if copy:
    train_df = train_df.copy()
    if test_df is not None:
      test_df = test_df.copy()
    if val_df is not None:
      val_df = val_df.copy()
  
  dfs = [df for df in [train_df, val_df, test_df] if df is not None]

  for df in dfs:
    df2id_df(df, cols, id_mappings, rvs_id_mappings)

  for src_col, same_col in same_col_pairs:
    id_mappings[same_col] = id_mappings[src_col]
  
  same_cols = [same_col for _, same_col in same_col_pairs]

  for df in dfs:
    df2id_df(df, same_cols, id_mappings, rvs_id_mappings)

  if copy:
    return tuple(dfs)

In [0]:
# columns to use
use_cols = ["pid", "userid", "category", "year", "artid", "albid", "traid"]

# other columns to use with the same mappings as the previous columns
use_col_pairs = [("year", "prev_year"),
                 ("artid", "prev_artid"),
                 ("albid", "prev_albid"),
                 ("traid", "prev_traid")]

# which columns are user/context columns - and known as query context
non_track_cols = ["category", "timestamp", "pid", "userid",
                  "prev_traid", "prev_albid", "prev_artid", "prev_year"]

# which columns are item columns - and known only as query seeds
track_cols = ["traid", "albid", "artid", "year"]

In [0]:
def _to_query_ctx(df):
  return df[non_track_cols]


def _to_query(concat_df):
  concat_df[track_cols] = concat_df[track_cols].fillna(0)
  concat_df[track_cols] = concat_df[track_cols].astype("int64", copy=False)
  return concat_df


def get_query_seeds(query_df):
  """
  Get the seed tracks of a query dataframe
  """
  return query_df[query_df["traid"] != 0]


def get_query_ctx(query_df):
  """
  Get the context of a query dataframe - 
  the actual target information without any target track information
  """
  return _to_query_ctx(query_df[query_df["traid"] == 0])


def _last_track_split(df, as_val):
  i = 2 if as_val else 1
  gp = df.groupby("pid")
  train_df = df[gp.cumcount(ascending=False) >= i]
  test_df = gp.nth(-i).reset_index()
  return train_df, test_df


def _users_split(df, n_test_users, seed):
  users = df["pid"].drop_duplicates()
  np.random.seed(seed)
  test_users = np.random.choice(users, n_test_users, replace=False)
  test_users_mask = df["pid"].isin(test_users)
  test_users_df = df[test_users_mask]
  train_users_df = df[~test_users_mask]
  return train_users_df, test_users_df


def _known_last_track_split(df, as_val, seed, n_test_users=None):
  train_df, test_df = _last_track_split(df, as_val)

  train_tracks = pd.Index(train_df["traid"].drop_duplicates())
  test_tracks = pd.Index(test_df["traid"].drop_duplicates())
  unknown_tracks = test_tracks.difference(train_tracks)
  unknown_mask = test_df["traid"].isin(unknown_tracks)
  unknown_pids = pd.Index(test_df[unknown_mask]["pid"])

  unknown_mask = df["pid"].isin(unknown_pids)
  known_df = df[~unknown_mask]
  unknown_df = df[unknown_mask]

  if n_test_users:
    train_users_df, test_users_df = _users_split(known_df, n_test_users, seed)
    train_df, test_df = _last_track_split(test_users_df, as_val)
    train_df = pd.concat([unknown_df, train_users_df, train_df], sort=False)
  else:
    train_df, test_df = _last_track_split(known_df, as_val)
    train_df = pd.concat([unknown_df, train_df], sort=False)

  train_df.sort_values("pid", inplace=True, kind="mergesort")

  return train_df, test_df


def _last_track_query(train_df, test_df, n_test_users=None):
  if n_test_users:
    test_pids = pd.Index(test_df["pid"])
    test_query_df = train_df[train_df["pid"].isin(test_pids)]
  else:
    test_query_df = train_df

  test_query_ctx_df = _to_query_ctx(test_df)
  test_query_df = pd.concat([test_query_df, test_query_ctx_df], sort=False)
  test_query_df = _to_query(test_query_df)

  return test_query_df


def last_track_split(df, as_val=False, n_test_users=None, seed=1):
  """
  Split the dataframe into train and validation or test dataframes.
  The test dataframe contains the last track of each or n_test_users random
  users, the validation dataframe the second-last. Only returns validation or 
  test users with known (second-)last track indices.

  Also returns a query dataframe corresponding to the validation or test
  dataframe.

  Columns are mapped to one-hot indices in the process. 
  """
  train_df, test_df = _known_last_track_split(df, as_val, seed, n_test_users)

  train_df, test_df = split_dfs2id_dfs(train_df, test_df=test_df, cols=use_cols,
                                       same_col_pairs=use_col_pairs)

  return train_df, test_df, _last_track_query(train_df, test_df, n_test_users)


def _seed_users_split(df, n_test_users, include_val, n_seed_tracks, seed):
  gp = df.groupby("pid")
  seed_df = gp.filter(lambda g: g.size > n_seed_tracks)
  non_seed_df = gp.filter(lambda g: g.size <= n_seed_tracks)

  train_df, test_df = _users_split(seed_df, n_test_users, seed)

  val_df = None
  if include_val:
    train_df, val_df = _users_split(train_df, n_test_users, seed)

  train_df = pd.concat([train_df, non_seed_df], sort=False)
  train_df.sort_values("pid", inplace=True, kind="mergesort")

  if include_val:
    return train_df, val_df, test_df
  else:
    return train_df, test_df


def _seed_query_split(df, n_seed_tracks):
  gp = df.groupby("pid", as_index=False)
  test_df = df[gp.cumcount() >= n_seed_tracks]
  query_df = gp.head(n_seed_tracks)
  query_ctx_df = _to_query_ctx(gp.nth(n_seed_tracks))
  query_df = _to_query(pd.concat([query_df, query_ctx_df], sort=False))
  query_df.sort_values("pid", inplace=True, kind="mergesort")
  return test_df, query_df


def users_split(df, n_test_users, include_val=False, n_seed_tracks=5, seed=1):
  """
  Split the dataframe into train, optional validation and test dataframes.
  The test and validation dataframe each contain n_test_users users and
  the train dataframe the other users. The validation and test dataframes
  each contain the users tracks after his/her first n_seed_tracks tracks.
  Only returns validation or test users with more than n_seed_tracks tracks.

  Also returns query dataframes corresponding to the validation and test
  dataframes.

  Columns are mapped to one-hot indices in the process,
  but train, validation and test dataframes have separate pid indices,
  all start from 0!
  """
  dfs = _seed_users_split(df, n_test_users, include_val, n_seed_tracks, seed)

  # pids are separate for each split - all start from 0, to different n
  dfs = (split_dfs2id_dfs(df, cols=["pid"])[0] for df in dfs)
  if include_val:
    train_df, val_df, test_df = dfs
  else:
    train_df, test_df = dfs
    val_df = None

  use_cols_no_pid = use_cols[1:]
  split_dfs2id_dfs(train_df, test_df=test_df, val_df=val_df,
                   cols=use_cols_no_pid, same_col_pairs=use_col_pairs,
                   copy=False)
  
  test_df, test_query_df = _seed_query_split(test_df, n_seed_tracks)
  val_query_df = None
  if include_val:
    val_df, val_query_df = _seed_query_split(val_df, n_seed_tracks)

  dfs = train_df, val_df, test_df, val_query_df, test_query_df
  return tuple(df for df in dfs if df is not None)

In [0]:
def to_names_index(df):
  df = df[["traid", "traname", "albname", "artname"]].drop_duplicates()
  return df.set_index("traid")


def with_names(df, names_index_df):
  return df.join(names_index_df, on="traid").drop(columns=["traid"])


def drop_names(dfs):
  for df in dfs:
    df.drop(columns=["traname", "albname", "artname"], inplace=True)

# Saving as CSV

In [0]:
train_df, test_df, test_query_df = last_track_split(df_small, n_test_users=250)
names_df = to_names_index(train_df)
drop_names([train_df, test_df, test_query_df])

train_df.head()

,category,pid,timestamp,userid,sid,traid,artid,albid,year,prev_traid,prev_albid,prev_artid,prev_year
42341,0,0,1998-01-30,0,0,0,0,0,0,5073,3453,1711,59
138375,0,0,1998-01-30,0,1,1,1,1,59,0,0,0,0
229198,0,0,1998-01-30,0,2,2,2,2,1,1,1,1,59
309912,0,0,1998-01-30,0,3,3,3,3,59,2,2,2,1
769326,0,0,1998-01-30,0,3,4,0,4,0,3,3,3,59


In [0]:
train_df.to_csv("AOTM-2011-small-train.csv")
test_df.to_csv("AOTM-2011-small-test.csv")
test_query_df.to_csv("AOTM-2011-small-test-query.csv")